In [52]:
import GPy
import string
import numpy as np
import pandas as pd
#import cPickle as pickle
from GPy.util import pca
from matplotlib import pyplot as plt
import torch
from sklearn import svm
%matplotlib inline

In [53]:
def get_data(data_type,file_num):

    if(data_type=='train'):
        data=pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_train_{file_num}.csv")
        #data=pd.read_csv(f"Data/ihdp_npci_train_{file_num}.csv",index_col=False)
    else:
        data = pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_test_{file_num}.csv")
        #data = pd.read_csv(f"Data/ihdp_npci_test_{file_num}.csv",index_col=False)

    x_data=pd.concat([data.iloc[:,0], data.iloc[:, 1:30]], axis = 1)
 
   
    y_data=data.iloc[:, 1]
    
    return x_data,y_data

def cal_pehe(y_1,y_0,test_x):
    #pehe...
    cate_pred=y_1-y_0
    #cate_pred=y1-y0
    cate_true=test_x.iloc[:,4]-test_x.iloc[:,3] #Hill's noiseless true values
    cate_err=np.mean( np.square( ( (cate_true) - (cate_pred) ) ) )
    return np.sqrt(cate_err)

In [54]:
def train_GPLVM(input_dim,Yn):
    input_dim = input_dim
    # define covariance function to be used
    kernel = GPy.kern.Linear(input_dim, ARD=True)
    # setup the GPLVM model
    m = GPy.models.GPLVM(Yn, input_dim=input_dim, kernel=kernel)
    # optimize for 1000 iterations
    m.optimize(messages=1, max_iters=1000) 
    return m

In [55]:
#fil_num=50
input_dim=70
pehe_error=[]
for i in range(1,10):
    x_data,y_data=get_data('train',i)
    test_x,test_y=get_data('test',i)
    x_train_sr=x_data[x_data['treatment']==1]
    y_train_sr=y_data[x_data['treatment']==1]
    x_train_tr=x_data[x_data['treatment']==0]
    y_train_tr=y_data[x_data['treatment']==0]
    Yn=x_data.iloc[:,5:30].to_numpy()
    labels=x_data.iloc[:,0].to_numpy()
    m=train_GPLVM(input_dim,Yn)
    #color=np.where(labels==1, 'Red','Green')
    ar=np.array(m.X)
    reg_0 = svm.SVR()
    reg_1=svm.SVR()
    embedded_data=np.concatenate((ar, labels.reshape(-1, 1)),axis=1)
    s_data=embedded_data[embedded_data[:,input_dim]==1]
    t_data=embedded_data[embedded_data[:,input_dim]==0]
    test_data=m.infer_newX(test_x.iloc[:,5:30].to_numpy())
    test_embd=np.array(test_data[0])
    reg_1.fit(s_data[:,0:input_dim], y_train_sr)
    y_1=reg_1.predict(test_embd)
    reg_0.fit(t_data[:,0:input_dim], y_train_tr)
    y_0=reg_0.predict(test_embd)
    pehe=cal_pehe(y_1,y_0,test_x)
    pehe_error.append(pehe)

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s19  0001   2.294265e+04   1.308345e+07 
    03s25  0063  -7.018035e+04   2.633211e+08 
    10s42  0219  -1.234846e+05   3.874991e+08 
    11s58  0245  -1.234874e+05   3.038783e+05 
Runtime:     11s58
Optimization status: Converged

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s14  0000   2.294026e+04           nan 
    00s18  0001   2.294026e+04   1.308950e+07 
    02s23  0042  -3.896623e+04   3.891657e+08 
    08s35  0170  -1.225399e+05   5.948616e+07 
    26s80  0519  -1.227388e+05   7.306017e+07 
    52s37  1002  -1.246046e+05   3.656849e+09 
Runtime:     52s37
Optimization status: Maximum number of f evaluations reached

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s16  0001   2.294675e+04   1.305913e+07 
    02s21  0043  -5.058911e+04   1.340663e+08 
    08s33  0183  -1.238960e+0

In [56]:
pehe_error

[1.0316941702327485,
 0.5833869485543499,
 0.9468282605089777,
 0.5276573485791505,
 0.9363706463241634,
 2.20324496343598,
 0.5475006238254314,
 0.4952081124080344,
 1.1219374977458885]

In [57]:
np.mean(pehe_error)

0.9326476190683027